# Regressions

## Setup

In [1]:
import numpy as np
import pandas as pd
import requests
import bokeh.plotting as bk
from bokeh.models import HoverTool, Label, BoxZoomTool, PanTool, ZoomInTool, ZoomOutTool, ResetTool

API_URL = 'https://api.iextrading.com/1.0'

## Request To API

In [2]:
res = requests.get(f'{API_URL}/stock/V/chart/5y')
data = res.json()
df = pd.DataFrame(data)
df.head(5)








,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap
0,0.507237,0.000000,0.985,52.0157,2013-12-18,52.0470,"Dec 18, 13",50.7234,51.4722,4036771,16147084,51.6479
1,0.176322,0.003389,0.339,52.1920,2013-12-19,52.6241,"Dec 19, 13",51.8200,51.9142,3350467,13401868,52.2965
2,-0.026570,0.002878,-0.051,52.1654,2013-12-20,52.7040,"Dec 20, 13",52.1485,52.3466,5816824,23267296,52.3282
3,0.497574,0.012444,0.954,52.6630,2013-12-23,52.8562,"Dec 23, 13",52.3055,52.4118,2003857,8015428,52.6210
4,0.169078,0.015695,0.321,52.8321,2013-12-24,53.0388,"Dec 24, 13",52.4190,52.7693,1301911,5207644,52.6998
